In [3]:
import requests
import pandas as pd

# Blizzard API credentials
CLIENT_ID = 'e979052095524920a21f128369de2043'  
CLIENT_SECRET = 'XUhr3ncH4s6vSJKB2Nf5ZQUHSg7zrh4r'  

# Function to get access token
def get_access_token(client_id, client_secret):
    response = requests.post(
        'https://eu.battle.net/oauth/token',
        data={'grant_type': 'client_credentials'},
        auth=(client_id, client_secret)
    )
    return response.json()['access_token']




In [78]:
access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)

def get_auctionable_items(region, realm_id, access_token, id_threshold=200000):
    items_list = []

    # Fetch auction data for the specified realm
    url = f'https://{region}.api.blizzard.com/data/wow/connected-realm/{realm_id}/auctions'
    params = {
        'namespace': f'dynamic-{region}',
        'locale': 'en_GB',
        'access_token': access_token
    }
    response = requests.get(url, params=params)
    auction_data = response.json()

    # Check if the auction data contains items
    if 'auctions' not in auction_data:
        print("No auction data found.")
        return pd.DataFrame()

    # Extract item IDs and filter those above the threshold
    for auction in auction_data['auctions']:
        item_id = auction['item']['id']
        buyout = auction.get('buyout')  # Total buyout price for the auction

        # Convert prices from copper to gold
        
        buyout_gold = buyout / 10000 if buyout is not None else None

        if item_id > id_threshold:
            items_list.append({
                'Item ID': item_id, 
                'Quantity': auction['quantity'], 
                'Buyout (Gold)': buyout_gold, 
            })

    # Return only the first 10 items for testing
    return items_list[:10]

In [79]:
def get_item_details(region, item_id, access_token):
    url = f'https://{region}.api.blizzard.com/data/wow/item/{item_id}'
    params = {
        'namespace': f'static-{region}',
        'locale': 'en_GB',
        'access_token': access_token
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        item_data = response.json()
        item_name = item_data.get('name', 'Unknown')
        item_quality = item_data.get('quality', {}).get('name', 'Unknown')  # Fetch the quality name of the item
        return item_name, item_quality
    return 'Unknown', 'Unknown'


In [80]:
# Fetch a sample of items from the EU region
region = 'eu'
realm_id = 1378  # Replace with the realm ID for Dun Modr or your target server
threshold = 200000  # Set your desired threshold for item IDs
auctionable_items_df = get_auctionable_items(region, realm_id, access_token, id_threshold=threshold)


In [81]:
auctionable_items_df

[{'Item ID': 224603, 'Quantity': 1, 'Buyout (Gold)': 1000.21},
 {'Item ID': 224603, 'Quantity': 1, 'Buyout (Gold)': 8185.0},
 {'Item ID': 224599, 'Quantity': 1, 'Buyout (Gold)': 751.35},
 {'Item ID': 219475, 'Quantity': 1, 'Buyout (Gold)': 460.0},
 {'Item ID': 224623, 'Quantity': 1, 'Buyout (Gold)': 1154.19},
 {'Item ID': 222835, 'Quantity': 1, 'Buyout (Gold)': 860.0},
 {'Item ID': 211619, 'Quantity': 1, 'Buyout (Gold)': 3057.72},
 {'Item ID': 224627, 'Quantity': 1, 'Buyout (Gold)': 9304.08},
 {'Item ID': 222839, 'Quantity': 1, 'Buyout (Gold)': 225.0},
 {'Item ID': 222475, 'Quantity': 1, 'Buyout (Gold)': 300.17}]

In [82]:
# Retrieve item names and qualities, and create a DataFrame for the first 10 items
items_with_names = []
for item in auctionable_items_df:
    item_name, item_quality = get_item_details(region, item['Item ID'], access_token)
    item['Item Name'] = item_name
    item['Item Quality'] = item_quality
    items_with_names.append(item)

# Create a DataFrame with the items and their details
items_df = pd.DataFrame(items_with_names)

# Display the filtered DataFrame to the user
print(items_df)

   Item ID  Quantity  Buyout (Gold)  \
0   224603         1        1000.21   
1   224603         1        8185.00   
2   224599         1         751.35   
3   219475         1         460.00   
4   224623         1        1154.19   
5   222835         1         860.00   
6   211619         1        3057.72   
7   224627         1        9304.08   
8   222839         1         225.00   
9   222475         1         300.17   

                                       Item Name Item Quality  
0                          Underscout's Trousers     Uncommon  
1                          Underscout's Trousers     Uncommon  
2                    Deep-Dweller's Shoulderpads     Uncommon  
3                       Tracker's Chitin Hauberk         Rare  
4                              Earthforged Cuffs     Uncommon  
5                      Pioneer's Perfected Cloak         Rare  
6  Schematic: Draconic Combatant's Plate Goggles     Uncommon  
7                                Arathi Arquebus     Uncom

In [83]:
items_df

,Item ID,Quantity,Buyout (Gold),Item Name,Item Quality
0,224603,1,1000.21,Underscout's Trousers,Uncommon
1,224603,1,8185.00,Underscout's Trousers,Uncommon
2,224599,1,751.35,Deep-Dweller's Shoulderpads,Uncommon
3,219475,1,460.00,Tracker's Chitin Hauberk,Rare
4,224623,1,1154.19,Earthforged Cuffs,Uncommon
5,222835,1,860.00,Pioneer's Perfected Cloak,Rare
6,211619,1,3057.72,Schematic: Draconic Combatant's Plate Goggles,Uncommon
7,224627,1,9304.08,Arathi Arquebus,Uncommon
8,222839,1,225.00,Pioneer's Perfected Mantle,Rare
9,222475,1,300.17,Dredger's Developed Helm,Rare


In [84]:

# Group by Item ID and Item Quality, then select the row with the minimum Buyout price for each group
cheapest_items_df = items_df.loc[items_df.groupby(['Item ID', 'Item Quality'])['Buyout (Gold)'].idxmin()]


In [85]:
cheapest_items_df

,Item ID,Quantity,Buyout (Gold),Item Name,Item Quality
6,211619,1,3057.72,Schematic: Draconic Combatant's Plate Goggles,Uncommon
3,219475,1,460.00,Tracker's Chitin Hauberk,Rare
9,222475,1,300.17,Dredger's Developed Helm,Rare
5,222835,1,860.00,Pioneer's Perfected Cloak,Rare
8,222839,1,225.00,Pioneer's Perfected Mantle,Rare
2,224599,1,751.35,Deep-Dweller's Shoulderpads,Uncommon
0,224603,1,1000.21,Underscout's Trousers,Uncommon
4,224623,1,1154.19,Earthforged Cuffs,Uncommon
7,224627,1,9304.08,Arathi Arquebus,Uncommon
